## spacy_text_classification : Exercise
In this exercise, you are going to classify whether a given text belongs to one of possible classes ['BUSINESS', 'SPORTS', 'CRIME'].

you are going to use spacy for pre-processing the text, convert text to numbers and apply different classification algorithms.
##About Data: News Category Classifier
This data consists of two columns. - Text - Category

1.   Text are the description about a particular topic.
2.   Category determine which class the text belongs to.
3.   Category determine which class the text belongs to.
4.   we have classes mainly of 'BUSINESS', 'SPORTS', 'CRIME'
     and comes under Multi-class classification Problem.





In [1]:
#import pandas library

import pandas as pd

#read the dataset "news_dataset.json" provided and load it into dataframe "df"

df = pd.read_json('/content/drive/MyDrive/Data Sets/news_dataset.json')

#print the shape of data
print(df.shape)

#print the top5 rows

df.head()

(7500, 2)


,text,category
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS
3,This Richard Sherman Interception Literally Sh...,SPORTS
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS


In [2]:
#check the distribution of labels
df.category.value_counts()

,count
category,
CRIME,2500
SPORTS,2500
BUSINESS,2500


In [ ]:
#Add the new column "label_num" which gives a unique number to each of these labels



#check the results with top 5 rows

## **Preprocess the text**

In [6]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [8]:
#use this utility function to preprocess the text
#1. Remove the stop words
#2. Convert to base form using lemmatisation

def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [9]:
#create a new column "preprocessed_text" which store the clean form of given text [use apply and lambda function]

df['preprocessed_text'] = df['text'].apply(lambda x: preprocess(x))

In [10]:
#print the top 5 rows
df.head()

,text,category,preprocessed_text
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME,Larry Nassar blame victim say victimize newly ...
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME,woman Beats Cancer die fall horse
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS,vegas taxpayer spend Record $ 750 million New ...
3,This Richard Sherman Interception Literally Sh...,SPORTS,Richard Sherman Interception literally shake W...
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS,7 thing totally kill Weed Legalization Buzz


## **Get the spacy embeddings for each preprocessed text**

In [11]:
#create a new column "vector" that store the vector representation of each pre-processed text

df['vector_text'] = df['preprocessed_text'].apply(lambda x: nlp(x).vector)

In [12]:
#print the top 5 rows
df.head()

,text,category,preprocessed_text,vector_text
0,"Larry Nassar Blames His Victims, Says He 'Was ...",CRIME,Larry Nassar blame victim say victimize newly ...,"[-0.5585511, -0.29323253, -0.9253956, 0.189389..."
1,"Woman Beats Cancer, Dies Falling From Horse",CRIME,woman Beats Cancer die fall horse,"[-0.73039824, -0.43196002, -1.2930516, -1.0628..."
2,Vegas Taxpayers Could Spend A Record $750 Mill...,SPORTS,vegas taxpayer spend Record $ 750 million New ...,"[-1.9413117, 0.121578515, -3.2996283, 1.511650..."
3,This Richard Sherman Interception Literally Sh...,SPORTS,Richard Sherman Interception literally shake W...,"[-1.4702771, -0.685319, 0.57398, -0.31135806, ..."
4,7 Things That Could Totally Kill Weed Legaliza...,BUSINESS,7 thing totally kill Weed Legalization Buzz,"[-1.037173, -1.9495698, -1.7179357, 1.2975286,..."


In [13]:
# Train-Test splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.vector_text, df.category, test_size= 0.2, random_state=42)

## **Reshape the X_train and X_test so as to fit for models**

In [14]:
import numpy as np

#reshapes the X_train and X_test using 'stack' function of numpy. Store the result in new variables "X_train_2d" and "X_test_2d"
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)


## **Attempt 1:**

use spacy glove embeddings for text vectorization.

use Decision Tree as the classifier.

print the classification report.

In [15]:

from sklearn.tree import DecisionTreeClassifier


#1. creating a Decision Tree model object

model = DecisionTreeClassifier()


#2. fit with all_train_embeddings and y_train
model.fit(X_train_2d, y_train)



#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = model.predict(X_test_2d)



#4. print the classfication report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    BUSINESS       0.73      0.73      0.73       508
       CRIME       0.72      0.68      0.70       510
      SPORTS       0.67      0.71      0.69       482

    accuracy                           0.71      1500
   macro avg       0.71      0.71      0.71      1500
weighted avg       0.71      0.71      0.71      1500



## **Attempt 2:**

use spacy glove embeddings for text vectorization.
use MultinomialNB as the classifier after applying the MinMaxscaler.
print the classification report.

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report



#doing scaling because Negative values will not pass into Naive Bayes models
scaler = MinMaxScaler()
X_train_2d = scaler.fit_transform(X_train_2d)
X_test_2d = scaler.transform(X_test_2d)


#1. creating a MultinomialNB model object
model1 = MultinomialNB()

#2. fit with all_train_embeddings(scaled) and y_train
model1.fit(X_train_2d, y_train)



#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = model1.predict(X_test_2d)



#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    BUSINESS       0.80      0.81      0.81       508
       CRIME       0.86      0.83      0.84       510
      SPORTS       0.81      0.82      0.81       482

    accuracy                           0.82      1500
   macro avg       0.82      0.82      0.82      1500
weighted avg       0.82      0.82      0.82      1500



## **Attempt 3:**

use spacy glove embeddings for text vectorization.
use KNeighborsClassifier as the classifier after applying the MinMaxscaler.
print the classification report.

In [18]:
from  sklearn.neighbors import KNeighborsClassifier


#1. creating a KNN model object

model2 = KNeighborsClassifier()



#2. fit with all_train_embeddings and y_train
model2.fit(X_train_2d, y_train)



#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = model2.predict(X_test_2d)



#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    BUSINESS       0.86      0.90      0.88       508
       CRIME       0.83      0.90      0.86       510
      SPORTS       0.91      0.79      0.85       482

    accuracy                           0.86      1500
   macro avg       0.87      0.86      0.86      1500
weighted avg       0.87      0.86      0.86      1500



## **Attempt 4:**

use spacy glove embeddings for text vectorization.
use RandomForestClassifier as the classifier after applying the MinMaxscaler.
print the classification report.

In [19]:
from sklearn.ensemble import RandomForestClassifier


#1. creating a Random Forest model object
model3 = RandomForestClassifier()

#2. fit with all_train_embeddings and y_train

model3.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred

y_pred = model3.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    BUSINESS       0.86      0.88      0.87       508
       CRIME       0.88      0.86      0.87       510
      SPORTS       0.87      0.87      0.87       482

    accuracy                           0.87      1500
   macro avg       0.87      0.87      0.87      1500
weighted avg       0.87      0.87      0.87      1500



## **Attempt 5:**

use spacy glove embeddings for text vectorization.
use GradientBoostingClassifier as the classifier after applying the MinMaxscaler.
print the classification report.

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

#1. creating a GradientBoosting model object
model4 = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
model4.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = model4.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    BUSINESS       0.90      0.91      0.91       508
       CRIME       0.90      0.87      0.89       510
      SPORTS       0.89      0.90      0.89       482

    accuracy                           0.89      1500
   macro avg       0.89      0.89      0.89      1500
weighted avg       0.89      0.89      0.89      1500



## **Print the confusion Matrix with the best model got**

In [1]:
#finally print the confusion matrix for the best model: GradientBoostingClassifier

from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)

import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(cm, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Truth')
figure = plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="g")
plt.show()




NameError: name 'y_test' is not defined